In [1]:
# --- Importing Libraries -------------------------------------------------------
import pandas as pd
import pickle
import dill
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import style
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.model_selection import train_test_split

# --- Matplotlib Settings -------------------------------------------------------
matplotlib.rc('figure', facecolor='k')
style.use('dark_background')

# --- Pandas Display Settings ---------------------------------------------------
pd.options.display.width = 113
pd.options.display.max_columns = 100
pd.options.display.colheader_justify='right'

# --- Importing Pickles ---------------------------------------------------------
def IMPORT_PICKLE(name):
    '''
    Returns pickled object within current file directory
    '''
    file_name = 'pickles/{}.pkl'.format(name)
    with open(file_name, 'rb') as my_file:
        return pickle.load(my_file)

# Dataset
X = IMPORT_PICKLE('minmax_dataset')
y = IMPORT_PICKLE('y')
feature_names = IMPORT_PICKLE('feature_names')

# Best classifier
model = IMPORT_PICKLE('best_model')

# Best classifier stats
stats = IMPORT_PICKLE('best_model_stats')

# Find thresholds function
with open('pickles/func_FIND_THRESHOLDS.pkl', 'rb') as myfile:
    FIND_THRESHOLDS = dill.load(myfile)